<a href="https://colab.research.google.com/github/invegat/DS-Unit-2-Sprint-2-Linear-Regression/blob/master/module3-gradient-descent/Gradient_Descent_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradient Descent Implementation Challenge!!

## Use gradient descent to find the optimal parameters of a **multiple** regression model. (We only showed an implementation for a bivariate model during lecture.)

A note: Implementing gradient descent in any context is not trivial, particularly the step where we calculate the gradient will change based on the number of parameters that we're trying to optimize for. You will need to research what the gradient of a multiple regression model looks like. This challenge is pretty open-ended but I hope it will be thrilling. Please work together, help each other, share resources and generally expand your understanding of gradient descent as you try and achieve this implementation. 

## Suggestions:

Start off with a model that has just two $X$ variables You can use any datasets that have at least two x variables. Potential candidates might be the blood pressure dataset that we used during lecture on Monday: [HERE](https://college.cengage.com/mathematics/brase/understandable_statistics/7e/students/datasets/mlr/excel/mlr02.xls) or any of the housing datasets. You would just need to select from them the two varaibles $x$ variables and one y variable that you want to work with that you most want to work with. 

Use Sklearn to find the optimal parameters of your model first. (like we did during the lecture.) So that you can compare the parameter estimates of your gradient-descent linear regression to the estimates of OLS linear regression. If implemented correctly they should be nearly identical.

Becoming a Data Scientist is all about striking out into the unknown, getting stuck and then researching and fighting and learning until you get yourself unstuck. Work together! And fight to take your own learning-rate fueled step towards your own optimal understanding of gradient descent! 


In [1]:
!pip install -U matplotlib

Requirement already up-to-date: matplotlib in /usr/local/lib/python3.6/dist-packages (3.0.2)


## Stretch Goals

If you happen upon the most useful resources for accomplishing this challenge first, I want you to spend time today studying other variations of Gradient Descent-Based Optimizers.

- Try and write a function that can perform gradient descent for arbitarily large (in dimensionality) multiple regression models. 
- Create a notebook for yourself exploring these topics
- How do they differ from the "vanilla" gradient descent we explored today
- How do these different gradient descent-based optimizers seek to overcome the challenge of finding the global minimum among various local minima?
- Write a blog post that reteaches what you have learned about these other gradient descent-based optimizers.

[Overview of GD-based optimizers](http://ruder.io/optimizing-gradient-descent/)

[Siraj Raval - Evolution of Gradient Descent-Based Optimizers](https://youtu.be/nhqo0u1a6fw)

In [1]:
%matplotlib inline
import matplotlib
matplotlib._version.sys.version

'3.6.7 (default, Oct 22 2018, 11:32:17) \n[GCC 8.2.0]'

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime
import math
from scipy.stats import mode
from sklearn.preprocessing import StandardScaler

In [3]:
xls_file = pd.ExcelFile('https://college.cengage.com/mathematics/brase/understandable_statistics/7e/students/datasets/mlr/excel/mlr02.xls')
xls_file

*** No CODEPAGE record, no encoding_override: will use 'ascii'


In [4]:
xls_file.sheet_names

['Sheet 1']

In [21]:
df = xls_file.parse('Sheet 1',names=['lbs','age', 'BP'])
df

,lbs,age,BP
0,132,52,173
1,143,59,184
2,153,67,194
3,162,73,211
4,154,64,196
5,168,74,220
6,137,54,188
7,149,61,188
8,159,65,207
9,128,46,167


In [22]:
scaler = StandardScaler()
std = scaler.fit_transform(df)
sdf = pd.DataFrame(std, columns = df.columns)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [23]:
sdf

,lbs,age,BP
0,-1.392392,-1.203018,-1.332610
1,-0.545762,-0.397519,-0.666305
2,0.223902,0.523051,-0.060573
3,0.916600,1.213479,0.969171
4,0.300869,0.177837,0.060573
5,1.378398,1.328551,1.514330
6,-1.007560,-0.972876,-0.424012
7,-0.083963,-0.167376,-0.424012
8,0.685700,0.292909,0.726878
9,-1.700257,-1.893446,-1.696050


In [0]:
def predict(features, weights):
  '''
  features - (200, 2)
  weights - (2, 1)
  predictions - (200,1)
  '''
  return np.dot(features,weights)

In [0]:
W_lbs = 0.0
W_age = 0.0

weights = np.array([
    [W_lbs],
    [W_age]
])
lr = .01

In [0]:
def bp_function(features, targets, weights):
    '''
    Features:(200,3)
    Targets: (200,1)
    Weights:(3,1)
    Returns 1D matrix of predictions
    '''
    N = len(targets)

    predictions = predict(features, weights)

    # Matrix math lets use do this without looping
    sq_error = (predictions - targets)**2

    # Return average squared error among predictions
    return 1.0/(2*N) * sq_error.sum()

In [0]:
def update_weights(features, targets, weights, lr):
    '''
    Features:(200, 2)
    Targets: (200, 1)
    Weights:(2, 1)
    '''
    predictions = predict(features, weights)

    #Extract our features
    x_lbs = features[:,0]
#     x_lbs = list(x_lbs)
    x_age = features[:,1]
#     x_age = list(x_age)
    print('x_lbs.shape', x_lbs.shape)
    print('(targets - predictions).shape',predictions.flatten(order='C').shape)
    r = np.subtract(targets, predictions)
    print('r.shape', r.shape)

    # Use matrix cross product (*) to simultaneously
    # calculate the derivative for each weight
    d_w_lbs = np.cross(x_lbs,np.array(list(targets)) - np.array(list(predictions)))
    d_w_age = np.cross(x_age,list(targets) - list(predictions))


    # Multiply the mean derivative by the learning rate
    # and subtract from our weights (remember gradient points in direction of steepest ASCENT)
    weights[0][0] -= (lr * np.mean(d_w_lbs))
    weights[1][0] -= (lr * np.mean(d_w_age))


    return weights

In [0]:
def train(lbs,age, BP, weights, learning_rate, iters):
    bp_history = []
    print(lbs.shape, age.shape, BP.shape,weights, learning_rate,iters)
    for i in range(iters):
        weights = update_weights(np.matrix([lbs, age]).T, BP, weights, learning_rate)

        #Calculate bp for auditing purposes
        bp = BP_function([lbs, age], BP, weights)
        bp_history.append(bo)

        # Log Progress
        if i % 10 == 0:
            print("iter: "+str(i) + " bp: "+str(bp))

    return weights, bp_history

In [153]:
train(df['lbs'],df['age'],df['BP'], weights, lr, 1000)

(11,) (11,) (11,) [[0.]
 [0.]] 0.01 1000
x_lbs.shape (11, 1)
(targets - predictions).shape (1, 11)


Exception: ignored